In [2]:
import keys
import pandas as pd
import requests
from pprint import pprint
import json
import numpy as np

In [4]:
df = pd.read_csv('merged_scraped_cleaned.csv')

In [5]:
df.shape

(1683, 7)

In [6]:
df.head()

,id,name,close_date,stage,address,lat,lng
0,0061a00000GcQdL,525 Water St SW- CMC Mgmt,3/8/18,Contract Sent,525 Water St SW Washington United States Distr...,38.876059,-77.020240
1,0061a00000Mi8dC,Bozzuto - NJ - Park + Garden,6/30/18,Invoice Sent,1450 Garden St Hoboken United States New Jerse...,40.754448,-74.028046
2,0061a00000M4PAU,Greystar- The Cameron,11/17/17,Deal Signed,"8710 Cameron St, Silver Spring, MD 20910",38.999289,-77.028885
3,0061a00000MgelJ,JLL- 2035 NW Front Ave,1/31/18,Proposal Sent,"1120 NW Couch St #500, Portland, OR 97209",45.537557,-122.688094
4,0061a00000KOF8z,International Monetary Fund- Spring Conference,1/31/18,Closed Won,"1900 Pennsylvania Ave NW, Washington, DC 20431",38.898940,-77.044101


In [ ]:
#add new columns for Transit Screen API info

In [11]:
df['ts_response'] = np.nan
df['mobility_score'] = np.nan
df['carshare'] = np.nan
df['bikeshare'] = np.nan
df['ridehailing'] = np.nan
df['masstransit'] = np.nan

In [12]:
base_url = 'http://mobilityscore-dev.transitscreen.io/api/v1/locations.json?&debug=true'
key = keys.ts

for i, row in df[(df['ts_response'].isnull()==True)
                 #& (df['id']=='0061a00000GcQdL') #for test
                ].iterrows():
    #print(row.id, row.name, row.lat, row.lng)
    try:
        url = base_url +'&key='+key+'&geojson=true&coordinates='+str(row.lat)+','+str(row.lng)
        #(print(url))
        response = requests.get(url) #wait up to 10 seconds
        data = json.loads(response.text)
        full_response = data['data']
        mobility_score = full_response['mobilityScore']['score']
        #print (mobility_score)
        raw_scores = full_response['mobilityScore']['rawScoreBreakdown']
        #print (raw_scores)
        carshare = raw_scores['carshare']
        bikeshare = raw_scores['bikeshare']
        ridehailing = raw_scores['ridehailing']
        masstransit = raw_scores['masstransit']
        #print(carshare, bikeshare, ridehailing, masstransit)
        df.loc[df['id']==row.id, 'ts_response'] = data
        df.loc[df['id']==row.id, 'mobility_score'] = mobility_score
        df.loc[df['id']==row.id, 'carshare'] = carshare
        df.loc[df['id']==row.id, 'bikeshare'] = bikeshare
        df.loc[df['id']==row.id, 'ridehailing'] = ridehailing
        df.loc[df['id']==row.id, 'masstransit'] = masstransit
    except:
        print('error on', i, raw_scores)
    if i%100 == 0:
        df.to_csv('model_data.csv', index=False)
        print (i, raw_scores)


0 {'carshare': 12.190236663104924, 'bikeshare': 21.71032692317476, 'ridehailing': 21, 'masstransit': 25.347331728747157}
100 {'carshare': 0, 'bikeshare': 0, 'ridehailing': 15, 'masstransit': 15.680790004933058}
200 {'carshare': 0, 'bikeshare': 36.442392006210916, 'ridehailing': 21, 'masstransit': 111.59136842875225}
400 {'carshare': 0, 'bikeshare': 0, 'ridehailing': 0, 'masstransit': 4.543233646460235}
800 {'carshare': 4.703640694113644, 'bikeshare': 19.639934533551557, 'ridehailing': 15, 'masstransit': 42.77287604036906}
1000 {'carshare': 0, 'bikeshare': 5.960644923715391, 'ridehailing': 15, 'masstransit': 59.23499401064204}
1100 {'carshare': 3.3940584335870363, 'bikeshare': 19.58650707290533, 'ridehailing': 13.5, 'masstransit': 163.6700110180089}
1300 {'carshare': 0, 'bikeshare': 6.46900269541779, 'ridehailing': 12.115384615384615, 'masstransit': 13.95485619648367}
1900 {'carshare': 2.656418139892391, 'bikeshare': 37.49438852852579, 'ridehailing': 21, 'masstransit': 130.1057092559521

In [13]:
df[df.id == '0061a00000GcQdL']

,id,name,close_date,stage,address,lat,lng,ts_response,mobility_score,carshare,bikeshare,ridehailing,masstransit
0,0061a00000GcQdL,525 Water St SW- CMC Mgmt,3/8/18,Contract Sent,525 Water St SW Washington United States Distr...,38.876059,-77.02024,{'data': {'mobilityScore': {'latitude': 38.876...,86.0,12.190237,21.710327,21.0,25.347332


In [14]:
df.to_csv('model_data.csv', index=False)